# Coursera Capstone
This notebook will be used for the capstone project.

In [39]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


#### Install BeautifulSoup and Import Libraries

In [41]:
# Install beautifulsoup4
# !conda install -c anaconda beautifulsoup4 --yes

# Import libraries
from bs4 import BeautifulSoup
import numpy as np

#### Donwload and Load Data

In [15]:
!wget -O 'toronto page.html' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
soup = BeautifulSoup(open("toronto page.html"))

--2018-12-14 18:28:51--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80280 (78K) [text/html]
Saving to: ‘toronto page.html’

toronto page.html   100%[=====================>]  78.40K  --.-KB/s   in 0.04s  

2018-12-14 18:28:52 (1.71 MB/s) - ‘toronto page.html’ saved [80280/80280]



#### Scrape Postal Code Table

In [71]:
table = soup.find('table', {'class': 'wikitable sortable'})
table_rows = table.find_all('tr')

res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res,columns = ["PostalCode","Borough","Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Clean Dataframe

**Clean 1** - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [83]:
flt = df['Borough']!='Not assigned' # Make a filter indicating if a borough is assigned
df_clean1 = df[flt] # Apply the filter
df_clean1.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


**Clean 2** - More than one neighborhood can exist in one postal code area. Combine these neighborhoods into one row.

In [96]:
series_clean2 = df_clean1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)) # Combine neighborhoods grouped by postal code
df_clean2 = pd.DataFrame(series_clean2).reset_index() # Change series to dataframe
df_clean2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Clean 3** - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [101]:
# For Neighborhood = Not assigned, make Neighborhood = Borough, otherwise keep Neighborhood
df_clean3 = df_clean2.copy()
df_clean3['Neighborhood'] = np.where(df_clean3['Neighborhood'] == 'Not assigned', df_clean3['Borough'], df_clean3['Neighborhood'])

In [115]:
# Compare change before and after
flt2 = df_clean2['Neighborhood'] == 'Not assigned'
print("Before clean3: \n",df_clean2[flt2].to_string(index=False),"\n \nAfter clean3: \n",df_clean3[flt2].to_string(index=False))

Before clean3: 
 PostalCode       Borough  Neighborhood
      M7A  Queen's Park  Not assigned 
 
After clean3: 
 PostalCode       Borough  Neighborhood
      M7A  Queen's Park  Queen's Park


In [122]:
# Show head of the final neighborhood dataframe
df_clean3.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Check the number of rows of the neighborhood dataframe.

In [119]:
df_clean3.shape

(103, 3)